In [90]:
import pathlib
import numpy as np
import pandas as pd

from utils import *
from viz import *
from task_system import TaskSystem
from policies.edf import EDF
from policies.fjp import FJP

import plotly.express as px
import plotly.graph_objects as go

import os
os.environ["IMAGEIO_FFMPEG_EXE"] = '/Users/abdulhakeemabdulrahman/homebrew/bin/ffmpeg'

import  moviepy.editor as mpy
import io 
from PIL import Image

In [91]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:
def run(filename, out_filename, policy, atomic=False, steps=None):
    sys = TaskSystem()
    sys.load_tasks(filename=filename)

    if steps is None:
        steps = sys.L

    for t in range(steps):
        index = policy(sys.tasks, atomic=atomic)

        sys.step(index)

    size = len(sys.trace)
    lines = []
    for start in range(0, size, sys.L):
        line = sys.trace[start: (start + sys.L)]
        line = [str(i) for i in line]
        line = ' '.join(line)
        lines.append(line)

        if len(lines) > 1 and lines[-1] == lines[0]:
            break
        
    with open(out_filename, 'w') as fp:
        for line in lines:
            fp.writelines(line + '\n')

    return out_filename
        

In [93]:
filename_set = f'data/set_fjp.txt'

name = 'fjp_preemptive'
filename_trace = f'data/trace_{name}.txt'

In [94]:
priorities = [1, 2, 3, 4]
policy = FJP(priorities)

In [95]:
run(filename_set, filename_trace, policy=policy)

'data/trace_fjp_preemptive.txt'

In [96]:
tasks = read_tasks(filename_set)
traces = read_traces(filename_trace)

In [97]:
output_dir = f'outputs/{name}'
pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

In [98]:
df_traces = process_traces(traces)
# df_traces

In [99]:
plot_polar(tasks, df_traces, 600, output_dir=None)

[0] 10 36.0
[266.66666667] [0.]
[3] 10 36.0
[-82.40453183] [253.61507101]
[4] 10 36.0
[-215.73786517] [156.74273394]
[1] 10 36.0
[215.73786517] [156.74273394]


In [59]:
clip = mpy.ImageSequenceClip(output_dir, fps=1)
clip.write_gif(f'outputs/{name}.gif')

MoviePy - Building file outputs/fjp_preemptive.gif with imageio.


In [48]:
plot_gantt(tasks, df_traces)